# TP2 - Modeling using docplex

## 1. The `docplex` python package

`DOcplex` is a python package developped by IBM &mdash; It provides easy-to-use API for IBM solvers Cplex and Cpoptimizer.

DOcplex documentation for mathematical programming can be found here: http://ibmdecisionoptimization.github.io/docplex-doc/#mathematical-programming-modeling-for-python-using-docplex-mp-docplex-mp

## 2. Solving TSP using `docplex`

### 2.1. TSP model using `docplex`

**Exercice:** Using `docplex`, create a model for the travelling salesman problem using the MTZ or Flow formulation and compare them.

In [1]:
from docplex.mp.model import Model
import tsp.data as data

distances = data.grid42
N = len(distances)

tsp = Model("TSP")
tsp.log_output = True

x = [tsp.binary_var_list(N, name=f"x{i}") for i in range(N)]
y = [tsp.continuous_var_list(N,name = f"y_{i}") for i in range(N)]

tsp.add_constraint(sum(y[1][j] for j in range(1,N))==1)

for i in range(N):
    tsp.add_constraint(sum(x[i][j] for j in range(N) if i!=j ) == 1 )
    
for i in range(N):
    tsp.add_constraint(sum(x[j][i] for j in range(N)if i!=j ) == 1 )
    
for i in range(1,N):
    tsp.add_constraint((sum(y[i][j] for j in range(N)))== sum(y[j][i] for j in range(N))+1)
    
for i in range(N):
    for j in range(N):
        tsp.add_constraint(y[i][j]<=N*x[i][j])

tsp.minimize(sum (sum(x[i][j]*distances[i][j] for i in range(N)) for j in range(N)))   
solution = tsp.solve()
print("z* =", solution.objective_value)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 42 rows and 83 columns.
MIP Presolve modified 40 coefficients.
Reduced MIP has 1848 rows, 3445 columns, and 10291 nonzeros.
Reduced MIP has 1722 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.31 ticks)
Found incumbent of value 3229.000000 after 0.01 sec. (11.54 ticks)
Probing fixed 0 vars, tightened 41 bounds.
Probing time = 0.02 sec. (8.08 ticks)
Cover probing fixed 0 vars, tightened 80 bounds.
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 40 rows and 40 columns.
MIP Presolve modified 121 coefficients.
Reduced MIP has 1808 rows, 3405 columns, and 10211 nonzeros.
Reduced MIP has 1722 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (7.11 ticks)
Probing time = 0.01 sec. (3.91 ticks)
Cover probing fixed 0 vars, tightened 240 bounds.
Clique table members: 8

The largest set of distances contains 42 nodes, and should be easily solved by `docplex`.

### 2.2. Generating random TSP instances

**Question:** What method could you implement to generate a realistic set of distances for $n$ customers?

**Exercice:** Implement the method proposed above and test it.

Pour générer un set de distance réaliste, nous utilisons la méthode np.random.rand qui génére un tableau de la taille que nous souhaitons, ici l'argument n pour le nombre de ligne et 2 pour représenter avoir les coordonnées d'une ville. Nous multiplions les valeurs par 100 car la randomisation les génères entre 0 et 1, donc pour obtenir des valeurs réalistes, multiplier par 100 semblent une bonne idée.
On crée ensuite une matrice de distance entre les différentes villes à l'aide de spicy.spatial.distance.distance_matrix

In [2]:
import numpy as np
from scipy.spatial import distance_matrix


def generate_distances(n: int):
    var = (np.random.rand(n,2)*100).astype(int)
    return distance_matrix(var,var,1)


from docplex.mp.model import Model

distances = generate_distances(42)
print(distances)

N = len(distances)

tsp = Model("TSP")
tsp.log_output = True


x = [tsp.binary_var_list(N, name=f"x{i}") for i in range(N)]
y = [tsp.continuous_var_list(N,name = f"y_{i}") for i in range(N)]

tsp.add_constraint(sum(y[1][j] for j in range(1,N))==1)

for i in range(N):
    tsp.add_constraint(sum(x[i][j] for j in range(N) if i!=j ) == 1 )
    
for i in range(N):
    tsp.add_constraint(sum(x[j][i] for j in range(N)if i!=j ) == 1 )
    
for i in range(1,N):
    tsp.add_constraint((sum(y[i][j] for j in range(N)))== sum(y[j][i] for j in range(N))+1)
    
for i in range(N):
    for j in range(N):
        tsp.add_constraint(y[i][j]<=N*x[i][j])

tsp.minimize(sum (sum(x[i][j]*distances[i][j] for i in range(N)) for j in range(N)))   

solution = tsp.solve()
print("z* =", solution.objective_value)

[[  0.  97.  38. ...  93. 102.  35.]
 [ 97.   0. 105. ...  54.  13. 132.]
 [ 38. 105.   0. ...  55. 110.  27.]
 ...
 [ 93.  54.  55. ...   0.  61.  78.]
 [102.  13. 110. ...  61.   0. 137.]
 [ 35. 132.  27. ...  78. 137.   0.]]
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 42 rows and 83 columns.
MIP Presolve modified 40 coefficients.
Reduced MIP has 1848 rows, 3445 columns, and 10291 nonzeros.
Reduced MIP has 1722 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.31 ticks)
Found incumbent of value 3016.000000 after 0.01 sec. (11.54 ticks)
Probing fixed 0 vars, tightened 41 bounds.
Probing time = 0.01 sec. (8.08 ticks)
Cover probing fixed 0 vars, tightened 80 bounds.
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 40 rows and 40 columns.
MIP Presolve modified 121 coefficients.
Reduced MIP has 1808 rows, 3405 columns, and 10211 n

## 3. Solving Warehouse Allocation using Benders decomposition with `docplex`

### 3.1. The warehouse problem

A company needs to supply a set of $n$ clients and needs to open new warehouses (from a
set of $m$ possible warehouses).
Opening a warehouse $j$ costs $f_j$ and supplying a client $i$ from a warehouse $j$ costs $c_{ij}$ per supply unit.
Which warehouses should be opened in order to satisfy all clients while minimizing the total cost?

### 3.2. Solving the warehouse problem with a single ILP

- $y_{j} = 1$ if and only if warehouse $j$ is opened.
- $x_{ij}$ is the fraction supplied from warehouse $j$ to customer $i$.

$
\begin{align}
  \text{min.} \quad & \sum_{i=1}^{n} \sum_{j=1}^{m} c_{ij} x_{ij} + \sum_{j=1}^{m} f_{j} y_{j} & \\
  \text{s.t.} \quad & \sum_{j=1}^{m} x_{ij} = 1, & \forall i \in\{1,\ldots,n\}\\
                    & x_{ij} \leq y_{j}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                    & y_{j} \in \left\{0,~1\right\}, & \forall j \in \left\{1,~\ldots,~m\right\}\\
                    & x_{ij} \geq 0, & \forall i \in \left\{1,~\ldots,~n\right\}, \forall j \in \left\{1,~\ldots,~m\right\}
\end{align}
$


**Exercice:** Implement the ILP model for the warehouse allocation problem and test it on the given instance.

In [3]:
from docplex.mp.model import Model

# We will start with a small instances with 3 warehouses and 3 clients:
N = 3
M = 3

# Opening and distribution costs:
f = [20, 20, 20]
c = [[15, 1, 2], [1, 16, 3], [4, 1, 17]]

wa = Model("Warehouse Allocation")
wa.log_output = True

# variables
y = wa.binary_var_list(M, name="Y")
x = [wa.integer_var_list(N, 0, 1, f"x{i}") for i in range(N)]

# contraintes
for j in range(M):
    wa.add_constraint(wa.sum(x[j])==1)

for j in range(M):
    for i in range(N):
        wa.add_constraint(x[i][j] <= y[j])

        
# Minimisation 
wa.minimize(wa.sum([wa.dot(x[i], c[i]) for i in range(N)]) + wa.dot(y,f))
solution = wa.solve()
print("z* =", solution.objective_value)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced MIP has 12 rows, 12 columns, and 27 nonzeros.
Reduced MIP has 12 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Found incumbent of value 82.000000 after 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 12 rows, 12 columns, and 27 nonzeros.
Reduced MIP has 12 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 12.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap



### 3.3. Benders' decomposition for the Warehouse Allocation problem

We are going to use Benders' decomposition to solve the Warehouse Allocation problem. 

#### Dual subproblem

$
\begin{align*}
\text{max.} \quad & \sum_{i=1}^{n} v_{i} - \sum_{i=1}^{n}\sum_{j=1}^{m} \bar{y}_{j} u_{ij} & \\
\text{s.t.} \quad & v_{i} - u_{ij} \leq c_{ij}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                  & v_{i} \in\mathbb{R},\ u_{ij} \geq 0 & \forall i \in\{1,\ldots,n\}, \forall j\in\{1,\ldots,m\}
\end{align*}
$

#### Master problem

$
\begin{align*}
  \text{min.} \quad & \sum_{j=1}^{m} f_j y_j + z & \\
  \text{s.t.} \quad & z \geq \sum_{i=1}^{n}v_i^p - \sum_{i=1}^{n} \sum_{j=1}^{m} u_{ij}^p y_j, & \forall p\in l_1\\
                    & 0 \geq \sum_{i=1}^{n}v_i^r - \sum_{i=1}^{n} \sum_{j=1}^{n} u_{ij}^r y_j, & \forall r\in l_2\\
                    & y_{j} \in\{0,1\}, & \forall j\in\{1,\ldots,m\}
\end{align*}
$

**Exercice:** Implement the method `create_master_problem` that creates the initial master problem (without feasibility or optimality constraints) for the warehouse allocation problem.

<div class="alert alert-info alert-block">

You can use `print(m.export_as_lp_string())` to display a textual representation of a `docplex` model `m`.
    
</div>

In [4]:
from docplex.mp.model import Model
from docplex.mp.linear import Var
from docplex.mp.constr import AbstractConstraint
from typing import List, Sequence, Tuple


def create_master_problem(
    N: int, M: int, f: Sequence[float], c: Sequence[Sequence[float]]
) -> Tuple[Model, Var, Sequence[Var]]:
    """
    Creates the initial Benders master problem for the Warehouse Allocation problem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        f: Array-like containing costs of opening warehouses.
        c: 2D-array like containing transport costs from client to warehouses.

    Returns:
        A 3-tuple containing the docplex problem, the z variable and the y variables.
    """

    wa = Model("Warehouse Allocation - Benders master problem")

    z = wa.integer_var()
    
    y = wa.binary_var_list(M)
    
    wa.minimize(wa.dot(y,f)+z)

    return wa, z, y


# Check your method:
wa, z, y = create_master_problem(N, M, f, c)
print(wa.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Warehouse Allocation - Benders master problem

Minimize
 obj: x1 + 20 x2 + 20 x3 + 20 x4
Subject To

Bounds
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1

Binaries
 x2 x3 x4

Generals
 x1
End



**Exercice:** Implement the method `add_optimality_constraints` that add optimality constraints to the Benders master problem. 

In [22]:
def add_optimality_constraint(
    N: int,
    M: int,
    wa: Model,
    z: Var,
    y: Sequence[Var],
    v: Sequence[float],
    u: Sequence[Sequence[float]],
) -> List[AbstractConstraint]:
    """
    Adds an optimality constraints to the given Warehouse Allocation model
    using the given optimal values from the Benders dual subproblem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        wa: The Benders master problem (docplex.mp.model.Model).
        z: The z variable of the master problem.
        y: The y variables of the master problem.
        v: The optimal values for the v variables of the Benders dual subproblem.
        u: The optimal values for the u variables of the Benders dual subproblem.

    Return: The optimality constraint added.
    """
    return wa.add_constraint(z >= sum(v[i] for i in range(N)) - sum(u[i][j]*y[j] for i in range(N) for j in range(M)))

**Exercice:** Implement the method `add_feasibility_constraints` that add feasibility constraints to the Benders master problem. 

In [21]:
def add_feasibility_constraints(
    N: int,
    M: int,
    wa: Model,
    z: Var,
    y: Sequence[Var],
    v: Sequence[float],
    u: Sequence[Sequence[float]],
) -> List[AbstractConstraint]:
    """
    Adds an optimality constraints to the given Warehouse Allocation model
    using the given optimal values from the Benders dual subproblem.

    Args:
      - N: Number of clients.
      - M: Number of warehouses.
      - wa: The Benders master problem (docplex.mp.model.Model).
      - z: The z variable of the master problem.
      - y: The y variables of the master problem.
      - v: The extreme rays for the v variables of the Benders dual subproblem.
      - u: The extreme rays for the u variables of the Benders dual subproblem.

    Returns:
        The feasibility constraint added.
    """
    return wa.add_constraint(0 >= sum(v[i] for i in range(N)) - sum(u[i][j]*y[j] for i in range(N) for j in range(M)))  

**Exercice:** Implement the method `create_dual_subproblem` that, given a solution `y` of the master problem, create the corresponding Benders dual subproblem.

$
\begin{align*}
\text{max.} \quad & \sum_{i=1}^{n} v_{i} - \sum_{i=1}^{n}\sum_{j=1}^{m} \bar{y}_{j} u_{ij} & \\
\text{s.t.} \quad & v_{i} - u_{ij} \leq c_{ij}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                  & v_{i} \in\mathbb{R},\ u_{ij} \geq 0 & \forall i \in\{1,\ldots,n\}, \forall j\in\{1,\ldots,m\}
\end{align*}
$

In [7]:
from docplex.mp.model import Model


def create_dual_subproblem(
    N: int, M: int, f: Sequence[float], c: Sequence[Sequence[float]], y: Sequence[int]
) -> Tuple[Model, Sequence[Var], Sequence[Sequence[Var]]]:
    """
    Creates a Benders dual subproblem for the Warehouse Allocation problem corresponding
    to the given master solution.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        f: Array-like containing costs of opening warehouses.
        c: 2D-array like containing transport costs from client to warehouses.
        y: Values of the y variables from the Benders master problem.

    Returns:
        A 3-tuple containing the docplex problem, the v variable and the u variables.
    """

    dsp = Model("Warehouse Allocation - Benders dual subproblem")

    # We disable pre-solve to be able to retrieve a meaningful status in the main
    # algorithm:
    dsp.parameters.preprocessing.presolve.set(0)

    v = dsp.continuous_var_list(N)
    u = [dsp.continuous_var_list(M, 0, name=f"x{i}") for i in range(N)] # 0 pour signifier que uij >= 0
    
    for i in range(N):
        for j in range(M):
            dsp.add_constraint(v[i]-u[i][j]<=c[i][j])
    dsp.maximize(dsp.sum(v)-dsp.sum([dsp.dot(u[i],y) for i in range(N)]))
    return dsp, v, u


# Check your method (assuming y = [1 1 1 ... 1]):
dsp, v, u = create_dual_subproblem(N, M, f, c, [1] * M)
print(dsp.export_as_lp_string())

-- cannot find parameters matching version: 22.1.1.0, using: 20.1.0.0
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Warehouse Allocation - Benders dual subproblem

Maximize
 obj: x1 + x2 + x3 - x0_0 - x0_1 - x0_2 - x1_0 - x1_1 - x1_2 - x2_0 - x2_1
      - x2_2
Subject To
 c1: x1 - x0_0 <= 15
 c2: x1 - x0_1 <= 1
 c3: x1 - x0_2 <= 2
 c4: x2 - x1_0 <= 1
 c5: x2 - x1_1 <= 16
 c6: x2 - x1_2 <= 3
 c7: x3 - x2_0 <= 4
 c8: x3 - x2_1 <= 1
 c9: x3 - x2_2 <= 17

Bounds
End



**Exercice:** Using the methods you implemented, write the Benders decomposition algorithm for the warehouse allocation problem.

<div class="alert alert-block alert-info">

The `get_extreme_rays` function can be used to retrieve the extreme rays associated with an unbounded solution of the dual subproblem.
    
</div>

<div class="alert alert-block alert-info">
    
You can use `model.get_solve_status()` to obtain the status of the resolution and compare it to members of `JobSolveStatus`:
    
```python
if model.get_solve_status() == JobSolveStatus.OPTIMAL_SOLUTION:
    pass
```
    
</div>

In [31]:
from docplex.mp.model import Model
from docplex.util.status import JobSolveStatus


def get_extreme_rays(
    N: int, M: int, model: Model, v: Sequence[Var], u: Sequence[Sequence[Var]]
) -> Tuple[Sequence[float], Sequence[Sequence[float]]]:
    """
    Retrieves the extreme rays associated to the dual subproblem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        model: The Benders dual subproblem model (docplex.mp.model.Model).
        v: 1D array containing the v variables of the subproblem.
        u: Either a 2D array of a tuple-index dictionary containing the u variables
            of the subproblem.

    Returns:
        A 2-tuple containing the list of extreme rays correspondig to v,
        and the 2D-list of extreme rays corresponding to u.
    """
    ray = model.get_engine().get_cplex().solution.advanced.get_ray()

    if isinstance(u, dict):

        def get_uij(i, j):
            return u[i, j]

    else:

        def get_uij(i, j):
            return u[i][j]

    return (
        [ray[v[i].index] for i in range(N)],
        [[ray[get_uij(i, j).index] for j in range(M)] for i in range(N)],
    )


# We will start with a small instances with 3 warehouses and 3 clients:
N = 3
M = 3

# Opening and distribution costs:
f = [20, 20, 20]
c = [[15, 1, 2], [1, 16, 3], [4, 1, 17]]

# We stop iterating if the new solution is less than epsilon
# better than the previous one:
epsilon = 1e-6

wa, z, y = create_master_problem(N, M, f, c)
previous_solution = 10000000

n = 0
while True:
    # Print iteration:
    n = n + 1
    print("Iteration {}".format(n))
    
    master_solution = wa.solve()


    dsp, v, u = create_dual_subproblem(N, M, f, c, master_solution.get_values(y))
    dual_subsolution = dsp.solve()    

    if (dsp.get_solve_status() != JobSolveStatus.UNBOUNDED_SOLUTION) and(abs(master_solution.get_value(z) - dual_subsolution.objective_value)<= epsilon):
        break
    if dsp.get_solve_status() == JobSolveStatus.UNBOUNDED_SOLUTION:
        ray_v, ray_u = get_extreme_rays(N, M, dsp, v, u)
        add_feasibility_constraints(N, M, wa, z, y, ray_v, ray_u)
    else:
        add_optimality_constraint(N, M, wa, z, y, dual_subsolution.get_values(v), [[u[i][j].solution_value for j in range(M)] for i in range(N)])
    
    previous_solution = master_solution.get_value(z)
print("\n")    
print("Done.")
print(master_solution.objective_value)

Iteration 1
-- cannot find parameters matching version: 22.1.1.0, using: 20.1.0.0
Iteration 2
-- cannot find parameters matching version: 22.1.1.0, using: 20.1.0.0
Iteration 3
-- cannot find parameters matching version: 22.1.1.0, using: 20.1.0.0
Iteration 4
-- cannot find parameters matching version: 22.1.1.0, using: 20.1.0.0
Iteration 5
-- cannot find parameters matching version: 22.1.1.0, using: 20.1.0.0


Done.
38.0


### 3.4. Generating instances for the Warehouse Allocation problem

**Exercice:** Using the TSP instances contained in `tsp.data` or the `generate_distances` method, create instances for the warehouse allocation problem with randomized opening costs.

In [39]:
from docplex.mp.model import Model
import numpy as np
import random
from scipy.spatial import distance_matrix


def generate_distances(n: int):
    var = (np.random.rand(n,2)*100).astype(int)
    return distance_matrix(var,var,1)


distance_1 = generate_distances(12)
distance_2 = generate_distances(10)


N = len(distance_1)
M = len(distance_2)

# Opening and distribution costs:
# f une liste de N valeurs
# c M listes de N valeurs

f = [random.randint(1,100) for _ in range (N)]
c = [[random.randint(1,100) for _ in range (N)] for _ in range(M)]

print("f")
print(f)
print("c")
print(c)
wa = Model("Warehouse Allocation")
wa.log_output = True

# variables
y = wa.binary_var_list(M, name="Y")
x = [wa.integer_var_list(N, 0, 1, f"x{i}") for i in range(N)]

# contraintes
for j in range(M):
    wa.add_constraint(wa.sum(x[j])==1)

for j in range(M):
    for i in range(N):
        wa.add_constraint(x[i][j] <= y[j])

        
# Minimisation 
wa.minimize(wa.sum([wa.dot(x[i], c[i]) for i in range(M)]) + wa.dot(y,f))
solution = wa.solve()
print("z* =", solution.objective_value)







f
[71, 95, 94, 52, 66, 77, 95, 26, 7, 15, 88, 95]
c
[[51, 8, 10, 20, 70, 40, 6, 78, 19, 73, 65, 61], [43, 28, 84, 9, 1, 20, 73, 31, 72, 11, 45, 62], [29, 78, 43, 86, 100, 95, 35, 38, 61, 84, 13, 66], [89, 73, 71, 93, 24, 100, 11, 96, 66, 34, 36, 36], [99, 49, 2, 4, 90, 94, 88, 10, 62, 99, 54, 16], [65, 56, 22, 38, 18, 16, 92, 15, 67, 16, 73, 32], [18, 88, 46, 16, 50, 15, 8, 55, 47, 74, 65, 6], [85, 4, 53, 84, 62, 30, 48, 13, 45, 18, 22, 93], [83, 60, 48, 29, 65, 44, 19, 82, 23, 43, 72, 81], [90, 47, 98, 22, 62, 69, 39, 4, 75, 45, 16, 7]]
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 461.000000 after 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 85 rows and 99 columns.
Reduced MIP has 45 rows, 55 columns, and 119 nonzeros.
Reduced MIP has 55 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.27 ticks)
Probing time = 0.00 sec. (0.03 ticks)
Tried aggregat

<div class="alert alert-block alert-danger"></div>